**Task 07: Querying RDF(s)**

In [15]:
#!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/AndreaCimminoArriaga/LinkedData2021-2022/main/Assignment4/"

Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [16]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
from rdflib.plugins.sparql import prepareQuery
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")
for s, p, o in g:
  print(s,p,o)

http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#Researcher http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#UPM http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#University
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#University http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [17]:
ns = Namespace("http://somewhere#")
print("Using RDFLib")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s)

print("Using SPARQL")

q1 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    ?s rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = {"ns" : ns, "rdfs" : RDFS}
)

for r in g.query(q1):
  print(r[0])


Using RDFLib
http://somewhere#Researcher
Using SPARQL
http://somewhere#Researcher


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [18]:
print("Using RDFLib")

for s, p, o in g.triples((None, RDF.type, None)):
  if o == ns.Person or (o, RDFS.subClassOf, ns.Person) in g:
    print(s)

print("Using SPARQL")

q2 = prepareQuery('''
  SELECT 
    ?s
  WHERE { 
    {
    ?o rdfs:subClassOf ns:Person.
    ?s rdf:type ?o.
    } UNION {
    ?s rdf:type ns:Person.
    }
  }
  ''',
  initNs = {"ns" : ns, "rdfs" : RDFS, "rdf" : RDF}
)

for r in g.query(q2):
  print(r[0])

Using RDFLib
http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
Using SPARQL
http://somewhere#JaneSmith
http://somewhere#SaraJones
http://somewhere#JohnSmith


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [24]:
print("Using RDFLib")

for s, p, o in g.triples((None, RDF.type, None)):
  if o == ns.Person or (o, RDFS.subClassOf, ns.Person) in g:
    print(f"Person {s}, with class {o}, has the following properties: \n")
    for su,pr,ob in g.triples((s, None, None)):
        print(pr)
    print()

print("Using SPARQL")

q3 = prepareQuery('''
  SELECT DISTINCT
    ?s ?p ?o ?ob
  WHERE { 
    {
    ?ob rdfs:subClassOf ns:Person.
    ?s rdf:type ?ob.
    ?s ?p ?o.
    } UNION {
    ?s rdf:type ns:Person.
    ?s rdf:type ?ob.
    ?s ?p ?o.
    }
  }
  ''',
  initNs = {"ns" : ns, "rdfs" : RDFS, "rdf" : RDF}
)
prop = dict()
clss = dict()
for s, p, o, ob in g.query(q3):
  prop.setdefault(s, set()).add(p)
  clss.setdefault(s, set()).add(ob)
print(f"Dictionary with all individuals and their properties:\n{prop}\nDictionary with all individuals and their class:\n{clss}")

Using RDFLib
Person http://somewhere#SaraJones, with class http://somewhere#Person, has the following properties: 

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family

Person http://somewhere#JohnSmith, with class http://somewhere#Person, has the following properties: 

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family

Person http://somewhere#JaneSmith, with class http://somewhere#Researcher, has the following properties: 

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/F